In [1]:
from datetime import datetime

import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from glob import glob
from multiprocessing import Pool
import time

In [2]:
# def format_convert(phaseinput, phaseoutput, nrms, ngap, nres, maxdep, maxdeperr, maxdiserr):
#     g = open(phaseoutput, 'w')
#     nn = 0
#     nres = float(nres)
#     nrms = float(nrms)
#     ngap = int(ngap)
#     maxdep = float(maxdep)
#     maxdeperr = float(maxdeperr)
#     maxdiserr = float(maxdiserr)
    
#     iok = 0  # Initialize iok outside of the loop

#     with open(phaseinput, "r") as f:
#         for line in f:
#             if len(line) == 180:
#                 iok = 0
#                 RMS = float(line[48:52]) / 100
#                 gap = int(line[42:45])
#                 dep = float(line[31:36])/100
#                 EZ = float(line[89:93])/100
#                 EH = float(line[85:89])/100

#                 if RMS <= nrms and gap <= ngap and dep <= maxdep and EZ <= maxdeperr and EH <= maxdiserr:
#                     nn = nn + 1
#                     year = int(line[0:4])
#                     mon = int(line[4:6])
#                     day = int(line[6:8])
#                     hour = int(line[8:10])
#                     min = int(line[10:12])
#                     sec = int(line[12:16])/100

#                     if line[18] == ' ':  # N
#                         lat = (float(line[16:18]) + float(line[19:23]) / 6000)
#                     else:
#                         lat = float(line[16:18]) + float(line[19:23])/6000 * (-1)

#                     if line[26] == 'E':
#                         lon = (float(line[23:26]) + float(line[27:31]) / 6000)
#                     else:
#                         lon = (float(line[23:26]) + float(line[27:31]) / 6000) * (-1)

#                     mag = float(line[123:126])/100
#                     g.write(
#                         '# {:4d} {:2d} {:2d} {:2d} {:2d} {:5.2f}  {:7.4f} {:9.4f}   {:5.2f} {:5.2f} {:5.2f} {:5.2f} {:5.2f} {:9d}\n'.format(
#                             year, mon, day, hour, min, sec, lat, lon, dep, mag, EH, EZ, RMS, nn))
#                     iok = 1
#             else:
#                 if iok == 1 and len(line) == 121:
#                     station = line[0:5]
#                     net = line[5:7]
#                     year1 = int(line[17:21])
#                     mon1 = int(line[21:23])
#                     day1 = int(line[23:25])
#                     hour1 = int(line[25:27])
#                     min1 = int(line[27:29])

#                     if year1 == year and mon1 == mon and day1 == day and hour1 == hour and min1 == min:
#                         sec_p =sec
#                         if line[13:15] == ' P' or line[13:15] == 'IP':
#                             P_residual = abs(int(line[34:38]) / 100)
#                             sec_p = int(line[29:34]) / 100
#                             if sec_p > sec and P_residual <= nres*RMS:
#                                 ppick = sec_p-sec
#                                 g.write('{:<5s}    {:8.3f}   1.000   P\n'.format(station, ppick))
#                                 # g.write('{}.{}    {:8.3f}   1.000   P\n'.format(net, station, ppick))

#                         if line[46:48] == ' S' or line[46:48] == 'ES':
#                             S_residual = abs(int(line[50:54]) / 100)
#                             sec_s = int(line[41:46]) / 100
#                             if sec_s > sec_p and S_residual <= nres * RMS:
#                                 spick = sec_s-sec
#                                 g.write('{:<5s}    {:8.3f}   1.000   S\n'.format(station, spick))
#                                 # g.write('{}.{}    {:8.3f}   1.000   S\n'.format(net, station, spick))
#     f.close()
#     g.close()

In [3]:
# if __name__ == '__main__':
#     # if len(sys.argv) != 9:
#     #     print('hypoinverse2hypoDD.py hypoOut.arc hypoDD.pha rms_threshold gap_threshold pick_rms maxdep maxdeperr maxdiserr')
#     #     sys.exit()
#     # format_convert(sys.argv[1],sys.argv[2],sys.argv[3],sys.argv[4],sys.argv[5],sys.argv[6],sys.argv[7],sys.argv[8])

#     input_file = 'results/hypoinput.arc'
#     output_file = 'results/hypoDD.pha'
#     rms_threshold=0.5 # in sec, events with rms larger than this will not be used
#     gap_threshold=300 # in deg., events with station gap larger than this will not be used
#     pick_nres=3       # if pick's residual larger than nres times event's rms
#                         # the pick will not be used
#     maxdep=20 # in km, events with larger depth will not be used (< dep in the timetable)
#     maxdep_err=5 # in km, events with larger depth uncertainty will not be used
#     maxdis_err=5 # in km, events with larger horizontal uncertainty will not be used
#     format_convert(input_file, output_file, rms_threshold, gap_threshold, pick_nres, maxdep, maxdep_err, maxdis_err)

In [4]:
def convert_stations():
    os.system("python convert_stations.py")
def convert_picks():

    picks = pd.read_csv("project_chile/gamma_picks.csv")
    events = pd.read_csv("project_chile/gamma_events.csv")

    if "file_index" in events.columns:
        events["match_id"] = events.apply(lambda x: f'{x["event_index"]}_{x["file_index"]}', axis=1)
        picks["match_id"] = picks.apply(lambda x: f'{x["event_index"]}_{x["file_index"]}', axis=1)
    elif "file_idx" in events.columns:
        events["match_id"] = events.apply(lambda x: f'{x["event_index"]}_{x["file_idx"]}', axis=1)
        picks["match_id"] = picks.apply(lambda x: f'{x["event_index"]}_{x["file_idx"]}', axis=1)
    else:
        events["match_id"] = events["event_index"]
        picks["match_id"] = picks["event_index"]
    events.sort_values(by="time", inplace=True, ignore_index=True)

    # %%
    # MAXEVENT = len(events)
    MAXEVENT = 2e4  ## segment by time
    MAXEVENT = len(events) // ((len(events) - 1) // MAXEVENT + 1) + 1

    # %% convert format
    idx = 0
    out_file = open(f"results/hypoDD.pha", "w")
    hypoDD_catalogs = []

    picks_by_event = picks.groupby("match_id").groups
    for i in tqdm(range(len(events))):
        if i % MAXEVENT == MAXEVENT - 1:
            out_file.close()
            idx = int(i // MAXEVENT)
            os.system(f"mkdir tmp_{idx:02d}")
            os.system(f"cp hypoDD.pha hypoDD.inp ph2dt.inp stations_hypoDD.dat ./tmp_{idx:02d}")
            # catalog = run_hypoDD(i, MAXEVENT)
            # hypoDD_catalogs.append(catalog)
            out_file = open("results/hypoDD.pha", "w")

        event = events.iloc[i]
        event_time = datetime.strptime(event["time"], "%Y-%m-%dT%H:%M:%S.%f")
        lat = event["latitude"]
        lng = event["longitude"]
        dep = event["depth_km"]
        mag = event["magnitude"]
        EH = 0
        EZ = 0
        #RMS = float(event["covariance"].split(",")[0])
        RMS = float(str(event["cov_time_amp"]).split(",")[0])
        if RMS > 10:
            RMS = 0
        year, month, day, hour, min, sec = (
            event_time.year,
            event_time.month,
            event_time.day,
            event_time.hour,
            event_time.minute,
            float(event_time.strftime("%S.%f")),
        )
        event_line = f"# {year:4d} {month:2d} {day:2d} {hour:2d} {min:2d} {sec:5.2f}  {lat:7.4f} {lng:9.4f}   {dep:5.2f} {mag:5.2f} {EH:5.2f} {EZ:5.2f} {RMS:5.2f} {i+1:9d}\n"
        out_file.write(event_line)

        picks_idx = picks_by_event[event["match_id"]]
        for j in picks_idx:
            pick = picks.iloc[j]
            network_code, station_code, comp_code, channel_code = pick['id'].split('.')
            phase_type = pick['type'].upper()
            phase_weight = pick['prob']
            pick_time = (datetime.strptime(pick["timestamp"], "%Y-%m-%dT%H:%M:%S.%f") - event_time).total_seconds()
            # if pick_time <= 0:
            #     continue
            pick_line = f"{station_code:<5s}    {pick_time:8.3f}   {phase_weight:5.4f}   {phase_type}\n"
            out_file.write(pick_line)

    out_file.close()
    idx = int(i // MAXEVENT)
    os.system(f"mkdir tmp_{idx:02d}")
    os.system(f"cp hypoDD.pha hypoDD.inp ph2dt.inp stations_hypoDD.dat ./tmp_{idx:02d}")

In [5]:
print(f"Start time: {time.asctime()}")
start_time = time.time()
#download_hypodd()
#download_test_data()
#convert_stations()
convert_picks()

Start time: Sat Jan 13 22:31:03 2024


100%|███████████████████████████████████████████████████████████████████████████| 12242/12242 [00:21<00:00, 561.35it/s]
